# Урок 3. Dataset, Dataloader, BatchNorm, Dropout, Оптимизация

In [7]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm_notebook
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 1. Создать Dataset для загрузки данных (используем только числовые данные)

In [1]:
!wget https://gist.githubusercontent.com/Yegorov/dc61c42aa4e89e139cd8248f59af6b3e/raw/20ac954e202fe6a038c2b4bb476703c02fe0df87/ya.py

--2022-04-01 20:47:32--  https://gist.githubusercontent.com/Yegorov/dc61c42aa4e89e139cd8248f59af6b3e/raw/20ac954e202fe6a038c2b4bb476703c02fe0df87/ya.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 723 [text/plain]
Saving to: ‘ya.py’

ya.py               100%[===================>]     723  --.-KB/s    in 0s      

2022-04-01 20:47:32 (24.3 MB/s) - ‘ya.py’ saved [723/723]



In [2]:
!chmod +x ya.py

In [3]:
!python ya.py https://disk.yandex.ru/d/mJeB-aaf8pIABA ./sample_data/data

--2022-04-01 20:49:49--  https://downloader.disk.yandex.ru/zip/0b2a8c964f6b569cedf92c766f201ef00c2e4095985add30c77ec10788b4fb60/624765e5/cVFYeFY3M1N6R3pwWVFWU2VQbW5Rb2xkQjVBSzh3SzhpbS9WaUM5SzRVanFLblZZQ0VKZU55RzhDVGZlU0k2NHEvSjZicG1SeU9Kb25UM1ZvWG5EYWc9PTo=?uid=0&filename=avito.zip&disposition=attachment&hash=qQXxV73SzGzpYQVSePmnQoldB5AK8wK8im/ViC9K4UjqKnVYCEJeNyG8CTfeSI64q/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&owner_uid=42105875&tknv=v2
Resolving downloader.disk.yandex.ru (downloader.disk.yandex.ru)... 77.88.21.127, 2a02:6b8::2:127
Connecting to downloader.disk.yandex.ru (downloader.disk.yandex.ru)|77.88.21.127|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://downloader-default2h.disk.yandex.net/rzip/0b2a8c964f6b569cedf92c766f201ef00c2e4095985add30c77ec10788b4fb60/624765e5/cVFYeFY3M1N6R3pwWVFWU2VQbW5Rb2xkQjVBSzh3SzhpbS9WaUM5SzRVanFLblZZQ0VKZU55RzhDVGZlU0k2NHEvSjZicG1SeU9Kb25UM1ZvWG5EYWc9PTo=?uid=0&filename=avito.zip&disposition=attachm

In [5]:
!unzip ./avito.zip

Archive:  ./avito.zip
  inflating: avito/periods_test.csv.zip  
  inflating: avito/periods_train.csv  
  inflating: avito/periods_train.csv.zip  
  inflating: avito/test.csv.zip      
  inflating: avito/test_active.csv.zip  
  inflating: avito/train.csv         
  inflating: avito/train.csv.zip     
  inflating: avito/train_active.csv.zip  


In [9]:
data = pd.read_csv('/content/avito/train.csv')
data.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [10]:
data.activation_date = pd.to_datetime(data.activation_date)
data['day_of_month'] = data.activation_date.apply(lambda x: x.day)
data['day_of_week'] = data.activation_date.apply(lambda x: x.weekday())

In [11]:
agg_cols = ['region', 'category_name',
            'city', 'user_type', 'parent_category_name']
for col in tqdm_notebook(agg_cols):
    le = LabelEncoder()
    le.fit(data[col])
    data[col] = le.transform(data[col])
    gp = data.groupby(col)['deal_probability']
    mean = gp.mean()
    data[col + '_deal_probability_avg'] = data[col].map(mean)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
data = data.drop(['user_id', 'description', 'image', 'parent_category_name',
                    'item_id', 'param_1', 'param_2', 'param_3', 'title', 'activation_date'], axis=1)

In [13]:
for col in data.columns:
    if data[col].isna().sum() > 0:
        data[col].fillna(data[col].median(), inplace=True)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 15 columns):
 #   Column                                     Non-Null Count    Dtype  
---  ------                                     --------------    -----  
 0   region                                     1503424 non-null  int64  
 1   city                                       1503424 non-null  int64  
 2   category_name                              1503424 non-null  int64  
 3   price                                      1503424 non-null  float64
 4   item_seq_number                            1503424 non-null  int64  
 5   user_type                                  1503424 non-null  int64  
 6   image_top_1                                1503424 non-null  float64
 7   deal_probability                           1503424 non-null  float64
 8   day_of_month                               1503424 non-null  int64  
 9   day_of_week                                1503424 non-null  int64  

In [15]:
train, test = train_test_split(data, test_size=0.25, random_state=13)
train.head(2)

,region,city,category_name,price,item_seq_number,user_type,image_top_1,deal_probability,day_of_month,day_of_week,region_deal_probability_avg,category_name_deal_probability_avg,city_deal_probability_avg,user_type_deal_probability_avg,parent_category_name_deal_probability_avg
31723,9,755,9,1300.0,6,1,1056.0,0.0,23,3,0.147066,0.178480,0.189069,0.149557,0.263336
929954,19,460,29,1300.0,16,1,399.0,0.0,28,1,0.122004,0.046447,0.123397,0.149557,0.075876


## 2. Обернуть его в Dataloader

In [16]:
class DFDataset(torch.utils.data.Dataset):
    def __init__(self, df, normalize=False, fit_scaler=False):
        self.df = df.copy()
        self.normalize = normalize
        self.scaler = MinMaxScaler()
        self.fit_scaler = fit_scaler
        self.sc_fl = 0

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        label = self.df.iloc[idx, -1:]
        if self.fit_scaler:
            self.scaler.fit(df.iloc[:, :-1])
            self.sc_fl = 1

        if self.normalize and sc_fl:
            df = scaler.transform(df)

        tensor = torch.FloatTensor(self.df.iloc[idx, 1:-1].values)
        label = torch.FloatTensor(label.values)

        return tensor, label

In [17]:
train_dataset = DFDataset(train)
test_dataset = DFDataset(test)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=2048,
                                           shuffle=True,
                                           num_workers=4)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=2048,
                                          shuffle=True,
                                          num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## 3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)

In [19]:
class FFNet(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FFNet, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = nn.Linear(input_dim, 5*hidden_dim)
        self.dp1 = nn.Dropout(0.40)

        self.bn2 = nn.BatchNorm1d(5*hidden_dim)
        self.fc2 = nn.Linear(5*hidden_dim, 2*hidden_dim)
        self.dp2 = nn.Dropout(0.15)

        self.bn4 = nn.BatchNorm1d(2*hidden_dim)
        self.fc4 = nn.Linear(2*hidden_dim, 1)

    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = F.tanh(x)
        x = self.dp1(x)

        x = self.bn2(x)
        x = self.fc2(x)
        x = F.tanh(x)
        x = self.dp2(x)

        x = self.bn4(x)
        x = self.fc4(x)
        x = F.sigmoid(x)

        return x

## 4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать

In [20]:
def rmsle_loss(y_pred, y_true):
    loss = torch.sqrt(torch.mean((torch.log(y_pred+1)-torch.log(y_true+1))**2))
    return loss

In [21]:
EPOCHES = 1
LR = 0.001

def train_loop(tr_dataloader, ev_dataloader, model, optimizer, history):

    size = len(tr_dataloader.dataset)
    for batch, (X, y) in enumerate(tr_dataloader):
        # Compute prediction and loss
        pred = model(X.to(device))
        loss = rmsle_loss(pred, y.to(device))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 50 == 0:
            current = batch * len(X)
            history['train'].append(loss)
            test_loss = eval_loop(ev_dataloader, model)
            history['eval'].append(test_loss)
            print(f"loss: {loss:>7f}, Avg test loss: {test_loss:>8f}  [{current:>5d}/{size:>5d}]")

    return history

def eval_loop(ev_dataloader, model):
    
    size = len(ev_dataloader.dataset)
    num_batches = len(ev_dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in ev_dataloader:
            pred = model(X.to(device))
            test_loss += rmsle_loss(pred, y.to(device))

    test_loss /= num_batches
    
    return test_loss

## 5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

### Adam

In [22]:
model = FFNet(13, 5).to(device)

In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [24]:
history_1 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(EPOCHES)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_1 = train_loop(train_loader,
                           test_loader,
                           model,
                           optimizer,
                           history_1
                          )
print("Done!")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


loss: 0.312399, Avg test loss: 0.311827  [    0/1127568]
loss: 0.279534, Avg test loss: 0.277836  [102400/1127568]
loss: 0.246090, Avg test loss: 0.245531  [204800/1127568]
loss: 0.192845, Avg test loss: 0.192475  [307200/1127568]
loss: 0.130438, Avg test loss: 0.127465  [409600/1127568]
loss: 0.074161, Avg test loss: 0.073180  [512000/1127568]
loss: 0.052097, Avg test loss: 0.050886  [614400/1127568]
loss: 0.046981, Avg test loss: 0.044832  [716800/1127568]
loss: 0.041794, Avg test loss: 0.041369  [819200/1127568]
loss: 0.038545, Avg test loss: 0.038668  [921600/1127568]
loss: 0.036476, Avg test loss: 0.036568  [1024000/1127568]
loss: 0.036126, Avg test loss: 0.034862  [642400/1127568]
Done!


### RMSProp

In [29]:
model = FFNet(13, 5).to(device)
optimizer = torch.optim.RMSprop(model.parameters(), lr=LR)
history_2 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(EPOCHES)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_2 = train_loop(train_loader,
                           test_loader,
                           model,
                           optimizer,
                           history_2
                          )
print("Done!")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


loss: 0.313713, Avg test loss: 0.301244  [    0/1127568]
loss: 0.209874, Avg test loss: 0.207529  [102400/1127568]
loss: 0.137300, Avg test loss: 0.133357  [204800/1127568]
loss: 0.081991, Avg test loss: 0.079714  [307200/1127568]
loss: 0.055459, Avg test loss: 0.055361  [409600/1127568]
loss: 0.046602, Avg test loss: 0.047937  [512000/1127568]
loss: 0.043642, Avg test loss: 0.044201  [614400/1127568]
loss: 0.040511, Avg test loss: 0.041515  [716800/1127568]
loss: 0.039544, Avg test loss: 0.039009  [819200/1127568]
loss: 0.034745, Avg test loss: 0.036727  [921600/1127568]
loss: 0.034949, Avg test loss: 0.034904  [1024000/1127568]
loss: 0.034530, Avg test loss: 0.033369  [642400/1127568]
Done!


### SGD

In [30]:
model = FFNet(13, 5).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
history_3 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(EPOCHES)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_3 = train_loop(train_loader,
                           test_loader,
                           model,
                           optimizer,
                           history_3
                          )
print("Done!")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


loss: 0.276550, Avg test loss: 0.276555  [    0/1127568]
loss: 0.276160, Avg test loss: 0.274809  [102400/1127568]
loss: 0.273227, Avg test loss: 0.273039  [204800/1127568]
loss: 0.269936, Avg test loss: 0.271341  [307200/1127568]
loss: 0.270137, Avg test loss: 0.269613  [409600/1127568]
loss: 0.267204, Avg test loss: 0.267927  [512000/1127568]
loss: 0.267274, Avg test loss: 0.266153  [614400/1127568]
loss: 0.266075, Avg test loss: 0.264425  [716800/1127568]
loss: 0.262572, Avg test loss: 0.262744  [819200/1127568]
loss: 0.262089, Avg test loss: 0.260950  [921600/1127568]
loss: 0.260916, Avg test loss: 0.259266  [1024000/1127568]
loss: 0.259292, Avg test loss: 0.257515  [642400/1127568]
Done!


Наблюдается самая высокая сходимость у RMSProp оптимизации, которая сравнима с Adam оптимизацией. В качестве вывода возможно сделать заключение, что RMSProp наиболее эффективный метод оптимизации, для данной задачи, тем не менее для других задач может быть более подходящий другой метод оптимизации, что необходимо учитывать в следующих исследованиях.